# Notebook dedicado a extraer las features de los Archivos H5:
- Se divide en varios intentos hasta conseguir el má eficiente

In [ ]:
import pandas as pd
import os
import glob
import hdf5_getters as GETTERS
import h5py
from pathlib import Path
from tqdm import tqdm
import os

In [12]:

# Ruta base para la letra A
dataset_path = r"C:\Dataset\msd_targz\A.tar\A"
file_paths = glob.glob(f"{dataset_path}\\**\\*.h5", recursive=True)  # Limitar a los primeros 1000 archivos

# O bien usando pathlib (más robusto en general)
# file_paths = list(Path(dataset_path).rglob("*.h5"))

#### 1

In [13]:
# Lista para los features
data = []

# Función para extraer


def extract_features(path):
    try:
        h5 = GETTERS.open_h5_file_read(path)

        features = {
            'song_id': GETTERS.get_song_id(h5).decode('utf-8'),
            'title': GETTERS.get_title(h5).decode('utf-8'),
            'artist': GETTERS.get_artist_name(h5).decode('utf-8'),
            'tempo': float(GETTERS.get_tempo(h5)),
            'key': float(GETTERS.get_key(h5)),
            'mode': float(GETTERS.get_mode(h5)),
            'time_signature': int(GETTERS.get_time_signature(h5)),
            'loudness': float(GETTERS.get_loudness(h5)),
            'danceability': float(GETTERS.get_danceability(h5))
        }

        h5.close()
        return features

    except Exception as e:
        print(f"Error con archivo {path}: {e}")
        return None


# Iterar sobre los archivos
for path in file_paths:
    result = extract_features(path)
    if result:
        data.append(result)

# Crear DataFrame
df_pd = pd.DataFrame(data)

# Guardar como parquet
df_pd.to_parquet(r"C:\Dataset\parquet\A.parquet", index=False)

print("Proceso completado con éxito.")

KeyboardInterrupt: 

#### 2

In [17]:


# Ruta base
dataset_path = r"C:\Dataset\msd_targz\A.tar\A"
file_paths = glob.glob(f"{dataset_path}\\**\\*.h5", recursive=True)

# Parámetros
batch_size = 1000
batch_data = []
batch_counter = 0
save_dir = r"C:\Dataset\parquet\batches"
os.makedirs(save_dir, exist_ok=True)

# Función para extraer features de un archivo .h5


def extract_features(path):
    try:
        h5 = GETTERS.open_h5_file_read(path)
        features = {
            'song_id': GETTERS.get_song_id(h5).decode('utf-8'),
            'title': GETTERS.get_title(h5).decode('utf-8'),
            'artist': GETTERS.get_artist_name(h5).decode('utf-8'),
            'tempo': float(GETTERS.get_tempo(h5)),
            'key': float(GETTERS.get_key(h5)),
            'mode': float(GETTERS.get_mode(h5)),
            'time_signature': int(GETTERS.get_time_signature(h5)),
            'loudness': float(GETTERS.get_loudness(h5)),
            'danceability': float(GETTERS.get_danceability(h5))
        }
        h5.close()
        return features
    except Exception as e:
        print(f"Error con archivo {path}: {e}")
        return None

# Procesamiento con barra de progreso
for i, path in enumerate(tqdm(file_paths, desc="Procesando archivos")):
    result = extract_features(path)
    if result:
        batch_data.append(result)

    if len(batch_data) >= batch_size:
        df_batch = pd.DataFrame(batch_data)
        output_path = os.path.join(
            save_dir, f"features_batch_{batch_counter}.parquet")
        df_batch.to_parquet(output_path, compression="snappy", index=False)
        print(f"Guardado: {output_path} ({len(df_batch)} canciones)")
        batch_data = []
        batch_counter += 1

# Guardar el último batch si queda algo
if batch_data:
    df_batch = pd.DataFrame(batch_data)
    output_path = os.path.join(
        save_dir, f"features_batch_{batch_counter}.parquet")
    df_batch.to_parquet(output_path, compression="snappy", index=False)
    print(f"Guardado (último): {output_path} ({len(df_batch)} canciones)")

Procesando archivos:   3%|▎         | 1009/39100 [00:18<12:08, 52.27it/s]

Guardado: C:\Dataset\parquet\batches\features_batch_0.parquet (1000 canciones)


Procesando archivos:   5%|▌         | 2010/39100 [00:36<11:12, 55.11it/s]

Guardado: C:\Dataset\parquet\batches\features_batch_1.parquet (1000 canciones)


Procesando archivos:   8%|▊         | 3009/39100 [00:55<10:41, 56.24it/s]

...

Guardado: C:\Dataset\parquet\batches\features_batch_37.parquet (1000 canciones)


Procesando archivos: 100%|█████████▉| 39008/39100 [12:25<00:01, 46.96it/s]

Guardado: C:\Dataset\parquet\batches\features_batch_38.parquet (1000 canciones)


Procesando archivos: 100%|██████████| 39100/39100 [12:27<00:00, 52.33it/s]

Guardado (último): C:\Dataset\parquet\batches\features_batch_39.parquet (100 canciones)


In [18]:
import pandas as pd
import glob
import os

# Ruta donde guardaste los batches
input_dir = r"C:\Dataset\parquet\batches"
output_file = r"C:\Dataset\parquet\features_A_final.parquet"

# Obtener todos los archivos .parquet de la carpeta
parquet_files = glob.glob(os.path.join(input_dir, "*.parquet"))

# Leer y concatenar todos los DataFrames
dataframes = []
for file in parquet_files:
    df = pd.read_parquet(file)
    dataframes.append(df)

# Concatenar todos en uno solo
df_final = pd.concat(dataframes, ignore_index=True)

# Guardar el resultado final
df_final.to_parquet(output_file, index=False, compression="snappy")

print(f"✅ Dataset final guardado en: {output_file}")
print(f"➡️ Número total de canciones: {len(df_final)}")

✅ Dataset final guardado en: C:\Dataset\parquet\features_A_final.parquet
➡️ Número total de canciones: 39100


In [20]:
import pandas as pd

# Ruta al archivo .parquet generado
parquet_path = r"C:\Dataset\parquet\A.parquet"

# Cargar el DataFrame desde Parquet
df = pd.read_parquet(parquet_path)

# Ver las primeras filas
print("Primeras filas del dataset:")
print(df.head())

# Ver información general
print("\nInformación general del DataFrame:")
print(df.info())

# Ver columnas
print("\nColumnas disponibles:")
print(df.columns.tolist())

Primeras filas del dataset:
              song_id                             title             artist  \
0  SOBLFFE12AF72AA5BA                            Scream       Adelitas Way   
1  SOQPWCR12A6D4FB2A3  A Poor Recipe For Civic Cohesion  Western Addiction   
2  SOMZWCG12A8C13C480                  I Didn't Mean To             Casual   
3  SOJDASC12A8C13EB49         The Lark In The Clear Air           Alquimia   
4  SOCIWDW12A8C13D406                         Soul Deep       The Box Tops   

     tempo  key  mode  time_signature  loudness  danceability  
0   99.944  1.0   1.0               4    -4.769           0.0  
1  125.475  7.0   1.0               4    -7.240           0.0  
2   92.198  1.0   0.0               4   -11.197           0.0  
3   41.279  2.0   1.0               4   -13.179           0.0  
4  121.274  6.0   0.0               4    -9.843           0.0  

Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39100 entries, 0 to 39099
Data co

#### 3

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# ---------------- CONFIGURACIÓN ----------------
dataset_path = r"C:\Dataset\msd_targz\A.tar\A"
output_dir = r"C:\Dataset\parquet\batches_A"
batch_size = 5000
num_threads = 12

# Crear carpeta si no existe
os.makedirs(output_dir, exist_ok=True)

# Obtener archivos .h5
file_paths = glob.glob(f"{dataset_path}\\**\\*.h5", recursive=True)

# ---------------- FUNCIÓN DE EXTRACCIÓN ----------------


def extract_features(path):
    try:
        h5 = GETTERS.open_h5_file_read(path)
        features = {
            'song_id': GETTERS.get_song_id(h5).decode('utf-8'),
            'title': GETTERS.get_title(h5).decode('utf-8'),
            'artist': GETTERS.get_artist_name(h5).decode('utf-8'),
            'tempo': np.float32(GETTERS.get_tempo(h5)),
            'key': np.int8(GETTERS.get_key(h5)),
            'mode': np.int8(GETTERS.get_mode(h5)),
            'time_signature': np.int8(GETTERS.get_time_signature(h5)),
            'loudness': np.float32(GETTERS.get_loudness(h5)),
            'danceability': np.float32(GETTERS.get_danceability(h5))
        }
        h5.close()
        return features
    except Exception as e:
        print(f"❌ Error con archivo {path}: {e}")
        return None


# ---------------- PROCESAMIENTO EN LOTES ----------------
for i in range(0, len(file_paths), batch_size):
    batch_paths = file_paths[i:i + batch_size]
    batch_data = []

    print(
        f"\n🔄 Procesando batch {i // batch_size + 1} ({len(batch_paths)} archivos)")

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(extract_features, path)
                   for path in batch_paths]
        for future in tqdm(as_completed(futures), total=len(futures), desc=f"Batch {i // batch_size + 1}"):
            result = future.result()
            if result:
                batch_data.append(result)

    if batch_data:
        df_batch = pd.DataFrame(batch_data)
        batch_file = os.path.join(
            output_dir, f"batch_{i // batch_size + 1}.parquet")
        df_batch.to_parquet(batch_file, index=False, compression="snappy")
        print(f"✅ Guardado: {batch_file} ({len(df_batch)} canciones)")


🔄 Procesando batch 1 (5000 archivos)


Batch 1: 100%|██████████| 5000/5000 [01:29<00:00, 56.08it/s] 


✅ Guardado: C:\Dataset\parquet\batches_A\batch_1.parquet (5000 canciones)

🔄 Procesando batch 2 (5000 archivos)


Batch 2: 100%|██████████| 5000/5000 [01:36<00:00, 51.83it/s] 


✅ Guardado: C:\Dataset\parquet\batches_A\batch_2.parquet (5000 canciones)

🔄 Procesando batch 3 (5000 archivos)


Batch 3: 100%|██████████| 5000/5000 [01:35<00:00, 52.40it/s]


✅ Guardado: C:\Dataset\parquet\batches_A\batch_3.parquet (5000 canciones)

🔄 Procesando batch 4 (5000 archivos)


Batch 4: 100%|██████████| 5000/5000 [01:23<00:00, 59.89it/s] 


✅ Guardado: C:\Dataset\parquet\batches_A\batch_4.parquet (5000 canciones)

🔄 Procesando batch 5 (5000 archivos)


Batch 5:  10%|█         | 503/5000 [00:09<01:24, 53.06it/s]


In [3]:
import pandas as pd
import glob
import os

# Ruta de los batches y output final
input_dir = r"C:\Dataset\parquet\batches_B"
output_file = r"C:\Dataset\parquet\B.parquet"

# Buscar todos los .parquet generados por batch
parquet_files = sorted(glob.glob(os.path.join(input_dir, "*.parquet")))

# Leer y concatenar
dataframes = []
for file in tqdm(parquet_files, desc="Unificando batches"):
    df = pd.read_parquet(file)
    dataframes.append(df)

df_final = pd.concat(dataframes, ignore_index=True)

# Guardar el único archivo final
df_final.to_parquet(output_file, index=False, compression="snappy")
print(
    f"\n✅ Dataset final guardado en: {output_file} ({len(df_final)} canciones)")

Unificando batches:   0%|          | 0/8 [00:00<?, ?it/s]

Unificando batches: 100%|██████████| 8/8 [00:00<00:00, 43.03it/s]


✅ Dataset final guardado en: C:\Dataset\parquet\B.parquet (38265 canciones)


#### 4

In [4]:
import os
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import multiprocessing

# ---------------- CONFIGURACIÓN ----------------
dataset_path = r"C:\Dataset\msd_targz\B.tar\B"
output_dir = r"C:\Dataset\parquet\batches_B"
batch_size = 5000
num_workers = min(12, multiprocessing.cpu_count())  # Usa hasta 12 cores

# Crear carpeta si no existe
os.makedirs(output_dir, exist_ok=True)

# Obtener archivos .h5
file_paths = glob.glob(f"{dataset_path}\\**\\*.h5", recursive=True)

# ---------------- FUNCIÓN DE EXTRACCIÓN ----------------


def extract_features(path):
    import hdf5_getters as GETTERS
    import numpy as np
    try:
        h5 = GETTERS.open_h5_file_read(path)
        features = {
            'song_id': GETTERS.get_song_id(h5).decode('utf-8'),
            'title': GETTERS.get_title(h5).decode('utf-8'),
            'artist': GETTERS.get_artist_name(h5).decode('utf-8'),
            'tempo': np.float32(GETTERS.get_tempo(h5)),
            'key': np.int8(GETTERS.get_key(h5)),
            'mode': np.int8(GETTERS.get_mode(h5)),
            'time_signature': np.int8(GETTERS.get_time_signature(h5)),
            'loudness': np.float32(GETTERS.get_loudness(h5)),
            'danceability': np.float32(GETTERS.get_danceability(h5))
        }
        h5.close()
        return features
    except Exception as e:
        print(f"❌ Error con archivo {path}: {e}")
        return None


# ---------------- PROCESAMIENTO EN LOTES ----------------
if __name__ == "__main__":
    for i in range(0, len(file_paths), batch_size):
        batch_paths = file_paths[i:i + batch_size]
        print(
            f"\n🔄 Procesando batch {i // batch_size + 1} ({len(batch_paths)} archivos)")

        with ProcessPoolExecutor(max_workers=num_workers) as executor:
            results = list(tqdm(executor.map(extract_features, batch_paths),
                                total=len(batch_paths),
                                desc=f"Batch {i // batch_size + 1}"))

        batch_data = [r for r in results if r]

        if batch_data:
            df_batch = pd.DataFrame(batch_data)
            batch_file = os.path.join(
                output_dir, f"batch_{i // batch_size + 1}.parquet")
            df_batch.to_parquet(batch_file, index=False, compression="snappy")
            print(f"✅ Guardado: {batch_file} ({len(df_batch)} canciones)")


🔄 Procesando batch 1 (5000 archivos)


BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore

In [5]:
print(multiprocessing.cpu_count())

8


In [ ]:
import pandas as pd
import glob
import os

# batches_dir = r"C:\Dataset\parquet\batches_E"
# output_file = r"C:\Dataset\parquet\E.parquet"

# batch_files = glob.glob(os.path.join(batches_dir, "*.parquet"))
# all_dfs = [pd.read_parquet(f) for f in batch_files]
# df_merged = pd.concat(all_dfs, ignore_index=True)
# df_merged.to_parquet(output_file, index=False, compression="snappy")

# print(f"🎉 Letra A completada → {output_file} ({len(df_merged)} canciones)")

for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
    batches_dir = f"C:\\Dataset\\parquet\\batches_{letter}"
    output_file = f"C:\\Dataset\\parquet\\{letter}.parquet"

    batch_files = glob.glob(os.path.join(batches_dir, "*.parquet"))
    all_dfs = [pd.read_parquet(f) for f in batch_files]
    df_merged = pd.concat(all_dfs, ignore_index=True)
    df_merged.to_parquet(output_file, index=False, compression="snappy")

    print(f"🎉 Letra {letter} completada → {output_file} ({len(df_merged)} canciones)")

🎉 Letra D completada → C:\Dataset\parquet\D.parquet (38825 canciones)


In [ ]:
import pandas as pd

# Ruta al archivo .parquet generado
parquet_path = r"C:\Dataset\parquet\C.parquet"

# Cargar el DataFrame desde Parquet
df = pd.read_parquet(parquet_path)

# Ver las primeras filas
print("Primeras filas del dataset:")
print(df.head())

# Ver información general
print("\nInformación general del DataFrame:")
print(df.info())

# Ver columnas
print("\nColumnas disponibles:")
print(df.columns.tolist())

Primeras filas del dataset:
              song_id                                   title  \
0  SOHODMI12AB0188D26  O Come All Ye Faithful (Album Version)   
1  SOPQYKD12A6701DECA                                   Games   
2  SOYXMUP12A81C1FD73                               Harmonize   
3  SOYTKKR12AF72A5DA5  Everybody Cares_ Everybody Understands   
4  SOMPLDW12A8C13883A                             Silver Star   

                   artist       tempo  key  mode  time_signature  loudness  \
0              Brenda Lee  103.718002    2     1               4   -13.411   
1  Big L / Sadat X / Guru   87.955002   11     0               4   -11.524   
2          Spirit Catcher  125.945000   11     0               3    -9.578   
3           Elliott Smith  116.129997    2     1               4    -8.396   
4              Chuck Loeb  169.977997    1     1               4    -9.517   

   danceability  
0           0.0  
1           0.0  
2           0.0  
3           0.0  
4           0.0  

Inf

#### Generar el dataset completo

In [1]:
import pandas as pd
import glob

# Ruta a todos los archivos .parquet por letra (A.parquet, ..., Z.parquet)
parquet_files = glob.glob(r"C:\Dataset\parquet\*.parquet")

# Leer y concatenar todos los .parquet
df = pd.concat([pd.read_parquet(f) for f in parquet_files], ignore_index=True)

# Guardar en un único archivo unificado (opcional)
df.to_parquet(r"C:\Dataset\parquet\full_dataset.parquet",
              index=False, compression='snappy')

# Mostrar el número total de canciones
print(f"🎵 Total de canciones en el dataset: {len(df)}")

🎵 Total de canciones en el dataset: 1000000
